# 05 - Degradation Model

## Overview
Train LightGBM model to predict tyre degradation.

In [4]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import matplotlib.pyplot as plt
from f1ts import config, io_flat, models_degradation, validation

## Load

In [5]:
features_dir = config.paths()['data_features']
deg_train = io_flat.read_parquet(features_dir / 'degradation_train.parquet')

✓ Loaded degradation_train.parquet: 2,533 rows, 32 cols
  Dtypes: {'session_key': dtype('O'), 'driver': dtype('O'), 'lap': dtype('int64'), 'lap_time_ms': dtype('int64'), 'sector1_ms': dtype('int64'), 'sector2_ms': dtype('int64'), 'sector3_ms': dtype('int64'), 'compound': dtype('O'), 'tyre_life': dtype('int64'), 'is_pit_lap': dtype('bool'), 'track_status': dtype('O'), 'stint_id': dtype('int64'), 'tyre_age_laps': dtype('int64'), 'air_temp': dtype('float64'), 'track_temp': dtype('float64'), 'humidity': dtype('float64'), 'rainfall': dtype('bool'), 'wind_speed': dtype('float64'), 'event_type': dtype('O'), 'duration_laps': dtype('float64'), 'lap_number': dtype('int64'), 'circuit_name': dtype('O'), 'pace_delta_roll3': dtype('float64'), 'pace_delta_roll5': dtype('float64'), 'deg_slope_last5': dtype('float64'), 'sector1_delta': dtype('int64'), 'sector2_delta': dtype('int64'), 'sector3_delta': dtype('int64'), 'pit_loss_s': dtype('float64'), 'sc_prob_next5': dtype('float64'), 'vsc_prob_next5': dt

## Transform: Train Model

In [9]:
# Ensure latest code is loaded after edits
import importlib
import f1ts.models_degradation as _md
importlib.reload(_md)
from f1ts import models_degradation
print("models_degradation reloaded:", models_degradation.__file__)

models_degradation reloaded: /workspaces/f1-track-strategy/src/f1ts/models_degradation.py


In [10]:
model, metrics = models_degradation.train_and_evaluate(deg_train)

Training samples: 1,782
Test samples: 751
Training degradation model...
Degradation Model Evaluation:
  MAE: 0.941s (940.7ms)
  RMSE: 1.132s (1132.0ms)
  Samples: 751


## Validate: Check Quality Gate

In [ ]:
try:
    validation.validate_degradation_model_quality(metrics['mae_s'])
except validation.ValidationError as e:
    print(f'⚠️  Quality gate not met (lenient threshold): {e}')

## Save

In [ ]:
models_dir = config.paths()['models']
io_flat.save_model(model, models_dir / 'degradation_v0.pkl')

metrics_dir = config.paths()['metrics']
io_flat.save_json(metrics, metrics_dir / 'degradation_metrics.json')

print('✓ Saved model and metrics')

## Repro Notes

- Trained LightGBM degradation model
- Split by session to avoid leakage
- Saved model and evaluation metrics